# Import Library 

In [1]:
import os
GPU = f'0'
os.environ['CUDA_VISIBLE_DEVICES']=GPU
import random
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflowjs as tfjs
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard
)
from tensorflow.keras.backend import set_learning_phase
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from config import *
from utils import helpers

#### Set Seed 

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

####  Config

In [3]:
IMAGE_PATH = f'../Datasets/HomePT/'
tfjs_target_dir = f'./checkpoints/best_model_js/'

batch_size = 16

classes = 3

learning_rate = 5e-5
wd = 0.0005
max_lr = 2e-3
min_lr = 5e-5
cycle_len = 20

EPOCHS = 100

#### Load Data 

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
)

val_datagen = ImageDataGenerator(
    rescale=1./255.,
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    IMAGE_PATH+f'train/',
    target_size=[480,480],
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    IMAGE_PATH+f'val/',
    target_size=[480,480],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1410 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


# Load Model 

In [6]:
# set_learning_phase(0)
# class EverySports(tf.keras.Model):
#     def __init__(self, model_variant):
#         # (RT, I, II, III or IV)
#         super(EverySports, self).__init__()
#         self.features = load_model(os.path.join('EfficientPose', 'keras', 'EfficientPose{0}.h5'.format(model_variant.upper())),
#                    custom_objects={'BilinearWeights': helpers.keras_BilinearWeights,
#                                    'Swish': helpers.Swish(helpers.eswish), 'eswish': helpers.eswish,
#                                    'swish1': helpers.swish1})
#         self.features.trainable = False
        
#         self.conv = Conv2D(1, 3, 1, padding='same', use_bias=False)
#         self.flatten = Flatten()
#         self.dense1 = Dense(1024)
#         self.dropout1 = Dropout(0.3)
#         self.dense2 = Dense(512)
#         self.dropout2 = Dropout(0.3)
#         self.dense3 = Dense(3)
    
#     def call(self, x):
#         x,_,_,_ = self.features(x)
#         x = self.conv(x)
#         # x = tf.keras.backend.sum(x, axis=-1)
        
#         x = self.flatten(x)
#         x = self.dense1(x)
#         x = self.dropout1(x)
#         x = tf.nn.relu(x)
        
#         x = self.dense2(x)
#         x = self.dropout2(x)
#         x = tf.nn.relu(x)
        
#         x = self.dense3(x)
#         x = tf.nn.softmax(x)
#         return x

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [7]:
# model = EverySports('III')

In [6]:
model_variant = 'III'
base_model = load_model(os.path.join('EfficientPose', 'keras', 'EfficientPose{0}.h5'.format(model_variant.upper())),
                   custom_objects={'BilinearWeights': helpers.keras_BilinearWeights,
                                   'Swish': helpers.Swish(helpers.eswish), 'eswish': helpers.eswish,
                                   'swish1': helpers.swish1})
base_model.trainable = False

In [7]:
model = tf.keras.Sequential([
    base_model,
    Conv2D(1, 3, 1, padding='same', use_bias=False),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

ValueError: All layers in a Sequential model should have a single output tensor. For multi-output layers, use the functional API.

# Compile

In [8]:
optimizer = tfa.optimizers.AdamW(learning_rate, wd)

model.compile(
    optimizer = optimizer,
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

#### Callbacks 

In [9]:
checkpoint_filepath = './checkpoints/best_model.h5'
mon = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False,
                                                monitor='val_accuracy', mode='max', save_best_only=True,
                                        verbose=1)

def trianfle_fn(x):
    return 1. / (2.**(x - 1))
clr_f = tfa.optimizers.CyclicalLearningRate(
    initial_learning_rate = max_lr,
    maximal_learning_rate = min_lr,
    step_size = cycle_len,
    scale_fn = trianfle_fn
)

clr = tf.keras.callbacks.LearningRateScheduler(clr_f)
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, 
                                          verbose=2, mode='auto', baseline=None, 
                                          restore_best_weights=True)

callbacks = [mon, clr, es]

# Training 

In [10]:
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
)

Epoch 1/100
89/89 [==============================] - ETA: 0s - loss: 0.0836 - accuracy: 0.9681
Epoch 00001: val_accuracy improved from -inf to 0.97778, saving model to ./checkpoints/best_model.h5
89/89 [==============================] - 19s 214ms/step - loss: 0.0836 - accuracy: 0.9681 - val_loss: 0.1233 - val_accuracy: 0.9778
Epoch 2/100
89/89 [==============================] - ETA: 0s - loss: 0.0075 - accuracy: 0.9972
Epoch 00002: val_accuracy improved from 0.97778 to 1.00000, saving model to ./checkpoints/best_model.h5
89/89 [==============================] - 17s 190ms/step - loss: 0.0075 - accuracy: 0.9972 - val_loss: 0.0051 - val_accuracy: 1.0000
Epoch 3/100
89/89 [==============================] - ETA: 0s - loss: 7.4379e-05 - accuracy: 1.0000
Epoch 00003: val_accuracy did not improve from 1.00000
89/89 [==============================] - 17s 187ms/step - loss: 7.4379e-05 - accuracy: 1.0000 - val_loss: 1.9602e-06 - val_accuracy: 1.0000
Epoch 4/100
89/89 [============================

# Inference 

In [11]:
file_path = f'../Datasets/HomePT/val/0/0.jpg'
img = cv2.imread(file_path)
h, w = img.shape[0], img.shape[1]
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
img = cv2.resize(img, dsize=(480, 480), interpolation=cv2.INTER_AREA)
img /= 255.
img = img[np.newaxis, ...]

output = model.predict(img)

# output = np.sum(output[0], -1)
# plt.imshow(output)

In [12]:
output.shape

(1, 3)

In [13]:
output

array([[9.9976629e-01, 5.1712132e-15, 2.3375664e-04]], dtype=float32)

# Convert tf model to tfjs 

In [14]:
tfjs.converters.save_keras_model(model, tfjs_target_dir)

NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.